In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mnist
import keras
import tensorflow as tf

#from keras.utils import plot_model
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils.vis_utils import plot_model
from keras.utils.vis_utils import model_to_dot
keras.utils.vis_utils.pydot = pydot
from sklearn.metrics import confusion_matrix
import tensorflow_datasets as tfds
from sklearn.datasets import load_boston

#from keras.models import Sequential
#from keras.layers import Dense
#from keras.utils import to_categorical
#from keras.datasets import mnist

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [38]:
#train_data, validation_data, test_data = tfds.load(
#    name="imdb_reviews", 
#    split=('train[:60%]', 'train[60%:]', 'test'),
#    as_supervised=True)

In [52]:
boston = load_boston()

In [53]:
#boston

In [54]:
boston_frame= pd.DataFrame(boston.data, columns=boston.feature_names)
boston_frame

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [55]:
train_data = boston.data

In [56]:
train_target = boston.target

In [57]:
train_data.shape

(506, 13)

In [58]:
train_target.shape

(506,)

In [59]:
train_data, test_data, train_target, test_target = train_test_split(train_data, train_target, test_size=0.20)

In [749]:
model_boston = Sequential([
    Dense(256, activation='relu', name='dense_1', input_shape=(13,)),
    Dense(128, activation='relu', name='dense_2'),
    Dense(64, activation='relu', name='dense_3'),
    Dense(64, activation='relu', name='dense_4'),
    Dense(1, activation='linear', name='dense_output')
])

model_boston.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                     loss='mse',
                     metrics=['mae'])   #mae

model_boston.summary()

Model: "sequential_112"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               3584      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_output (Dense)         (None, 1)                 65        
Total params: 48,961
Trainable params: 48,961
Non-trainable params: 0
_________________________________________________________________


In [748]:
#plot_model(model_boston, to_file='model.png')

In [720]:
model_boston.fit(
    train_data,
    train_target,
    epochs=1000,
    batch_size=1024,
    validation_split=0.05,
    verbose = 0
)

In [721]:
model_boston.evaluate(test_data, test_target)

4/4 [==============================] - 0s 3ms/step - loss: 24.5133 - mae: 3.1189


[24.513307571411133, 3.11893367767334]

In [722]:
predictions = model_boston.predict(test_data)

In [723]:
r2_score(test_target, predictions)

0.7624528682883116

In [724]:
#                                 mse        mae        r2
#
#    
# 2 слоя  512                  71.7633     5.5250     0.30457
# 3 слоя  512,512              70.7407     5.3534     0.31448
# 3 слоя  256,128              67.8120     5.3113     0.34286
# 4 слоя  512,256,128          74.8170     5.5993     0.27498
# 4 слоя  256,128,64           69.5151     5.3879     0.32636
# 5 слоев 512,256,128,64       64.9211     5.1276     0.37087
# 5 слоев 256,128,64,64        59.4194     4.7619     0.42419
# sigmoid                      259.502     12.725    -1.5147  возврат к relu
# nadam                        84.8453     5.4009     0.17780 возврат к adam
# learning_rate=0.0001         78.3092     5.7001     0.24114
# learning_rate=0.01           57.3536     4.6723     0.44421
# learning_rate=0.001          70.9673     5.3043     0.31228 возврат к 0.01
# 500 эпох                     29.6913     3.5564     0.71227
# 1000 эпох                    28.2400     3.0923     0.72633
# val_split 0.1                26.4181     3.3664     0.74399
# val_split 0.05               24.5133     3.1189     0.76245
    
    

In [727]:
train_data_1, test_data_1, train_target_1, test_target_1 = train_test_split(train_data, train_target, test_size=0.50)

In [729]:
# создание тензора с максимальными значениями
max_inp=tf.keras.layers.maximum([train_data_1,test_data_1])

In [733]:
train_data_1

array([[4.7547e-01, 0.0000e+00, 9.9000e+00, ..., 1.8400e+01, 3.9623e+02,
        1.2730e+01],
       [3.5114e-01, 0.0000e+00, 7.3800e+00, ..., 1.9600e+01, 3.9690e+02,
        7.7000e+00],
       [3.0347e-01, 0.0000e+00, 7.3800e+00, ..., 1.9600e+01, 3.9690e+02,
        6.1500e+00],
       ...,
       [2.1870e-02, 6.0000e+01, 2.9300e+00, ..., 1.5600e+01, 3.9337e+02,
        5.0300e+00],
       [8.8290e-02, 1.2500e+01, 7.8700e+00, ..., 1.5200e+01, 3.9560e+02,
        1.2430e+01],
       [2.3004e+00, 0.0000e+00, 1.9580e+01, ..., 1.4700e+01, 2.9709e+02,
        1.1100e+01]])

In [734]:
test_data_1

array([[4.54400e-02, 0.00000e+00, 3.24000e+00, ..., 1.69000e+01,
        3.68570e+02, 9.09000e+00],
       [1.58760e-01, 0.00000e+00, 1.08100e+01, ..., 1.92000e+01,
        3.76940e+02, 9.88000e+00],
       [1.44550e-01, 1.25000e+01, 7.87000e+00, ..., 1.52000e+01,
        3.96900e+02, 1.91500e+01],
       ...,
       [1.05393e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.86850e+02, 6.58000e+00],
       [6.14700e-01, 0.00000e+00, 6.20000e+00, ..., 1.74000e+01,
        3.96900e+02, 7.60000e+00],
       [1.43337e+01, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.83320e+02, 1.31100e+01]])

In [735]:
max_inp

<tf.Tensor: shape=(202, 13), dtype=float32, numpy=
array([[4.75470e-01, 0.00000e+00, 9.90000e+00, ..., 1.84000e+01,
        3.96230e+02, 1.27300e+01],
       [3.51140e-01, 0.00000e+00, 1.08100e+01, ..., 1.96000e+01,
        3.96900e+02, 9.88000e+00],
       [3.03470e-01, 1.25000e+01, 7.87000e+00, ..., 1.96000e+01,
        3.96900e+02, 1.91500e+01],
       ...,
       [1.05393e+00, 6.00000e+01, 8.14000e+00, ..., 2.10000e+01,
        3.93370e+02, 6.58000e+00],
       [6.14700e-01, 1.25000e+01, 7.87000e+00, ..., 1.74000e+01,
        3.96900e+02, 1.24300e+01],
       [1.43337e+01, 0.00000e+00, 1.95800e+01, ..., 2.02000e+01,
        3.83320e+02, 1.31100e+01]], dtype=float32)>

In [ ]:
#tf.keras.layers.experimental.preprocessing.Normalization(
#    axis=-1, dtype=None, mean=None, variance=None, **kwargs)

In [768]:
model_boston_1 = Sequential([
    Dense(256, activation='relu', name='dense_1', input_shape=(13,)),
    tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True),
    Dense(128, activation='relu', name='dense_2'),
    Dense(64, activation='relu', name='dense_3'),
    Dense(64, activation='relu', name='dense_4'),
    Dense(1, activation='linear', name='dense_output')
])

model_boston_1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                     loss='mse',
                     metrics=['mae'])   #mae

model_boston_1.summary()

Model: "sequential_117"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               3584      
_________________________________________________________________
layer_normalization_7 (Layer (None, 256)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_output (Dense)         (None, 1)                 65        
Total params: 49,473
Trainable params: 49,473
Non-trainable params: 0
________________________________________________

In [769]:
#model_boston_1.add(tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True))
#model_boston_1.summary()

In [770]:
model_boston_1.fit(
    train_data,
    train_target,
    epochs=1000,
    batch_size=1024,
    validation_split=0.05,
    verbose = 0
)

In [771]:
model_boston.evaluate(test_data, test_target)

4/4 [==============================] - 0s 4ms/step - loss: 198.2421 - mae: 10.7881


[198.24209594726562, 10.78814697265625]

In [772]:
predictions = model_boston.predict(test_data)

In [773]:
r2_score(test_target, predictions)

-0.9210726838477044

In [829]:
train_data_films = pd.read_csv('/home/tefi/Homework/Neural_network/IMDB Dataset.csv')

In [830]:
train_data_films

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [831]:
train_data_films.loc[train_data_films['sentiment'] == 'positive', 'sentiment'] = 1
train_data_films.loc[train_data_films['sentiment'] == 'negative', 'sentiment'] = 0

In [832]:
train_target_films = train_data_films['sentiment']
train_target_films

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: object

In [833]:
train_data_films = train_data_films['review'] 
train_data_films

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [835]:
train_data_films = train_data_films.values

In [836]:
train_target_films = train_target_films.values

In [837]:
train_data_films.shape

(50000,)

In [838]:
train_target_films.shape

(50000,)

In [839]:
train_target_films

array([1, 1, 1, ..., 0, 0, 0], dtype=object)

In [802]:
train_data_films_1, test_data_films_1, train_target_films_1, test_target_films_1 = train_test_split(train_data_films, train_target_films, test_size=0.2)

In [846]:
model_films = Sequential([
    Dense(256, activation='relu', name='dense_1', input_shape=(1,)),
    Dense(128, activation='relu', name='dense_2'),
    Dense(64, activation='relu', name='dense_3'),
    Dense(64, activation='relu', name='dense_4'),
    Dense(1, activation='sigmoid', name='dense_output')
])

In [847]:
model_films.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

model_films.summary()

Model: "sequential_123"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_output (Dense)         (None, 1)                 65        
Total params: 45,889
Trainable params: 45,889
Non-trainable params: 0
_________________________________________________________________


In [848]:
#plot_model(model_films, to_file='model.png')

In [849]:
train_data_films_1.shape

(40000,)

In [850]:
train_target_films_1.shape

(40000,)

In [ ]:
model_films.fit(
    train_data_films_1,
    train_target_films_1,
    epochs=20,
    batch_size=1024,
    verbose = 0
)


In [ ]:
model_films.evaluate(train_data_films_1, train_target_films_1)

In [ ]:
predictions = model_films.predict(train_data_films_1)